In [161]:
import pandas as pd
import numpy as np

In [162]:
from api.constants_test_1 import ACCOUNT_ID, API_KEY, OANDA_URL
from api.OandaApi import OandaApi
from utils.heiken_ashi import ohlc_to_heiken_ashi

In [163]:
api = OandaApi(ACCOUNT_ID, API_KEY, OANDA_URL)

In [164]:
candles = api.get_candles_df("USD_JPY", count=100, granularity="D")
candles = candles[["time", "mid_c", "mid_o", "mid_h", "mid_l"]].copy()
candles = candles.iloc[0:83].copy()
heiken_ashi = ohlc_to_heiken_ashi(candles)

heiken_ashi.tail(30)


candles (83, 5)


,time,ha_open,ha_high,ha_low,ha_close,ha_green,ha_streak,ha_open_at_extreme
53,2025-03-27 21:00:00+00:00,150.390785,151.214000,149.684000,150.44875,1,6,0
54,2025-03-30 21:00:00+00:00,150.419768,150.419768,148.702000,149.69175,0,-1,1
55,2025-03-31 21:00:00+00:00,150.055759,150.141000,148.977000,149.67450,0,-2,0
56,2025-04-01 21:00:00+00:00,149.865129,150.492000,149.102000,149.62525,0,-3,0
57,2025-04-02 21:00:00+00:00,149.745190,149.745190,145.190000,147.45475,0,-4,1
58,2025-04-03 21:00:00+00:00,148.599970,148.599970,144.552000,146.26575,0,-5,1
59,2025-04-06 21:00:00+00:00,147.432860,148.151000,144.820000,146.58625,0,-6,0
60,2025-04-07 21:00:00+00:00,147.009555,148.124000,145.966000,147.03375,1,1,0
61,2025-04-08 21:00:00+00:00,147.021652,148.284000,143.990000,146.55375,0,-1,0
62,2025-04-09 21:00:00+00:00,146.787701,147.800000,144.014000,146.01500,0,-2,0


In [165]:
if heiken_ashi.iloc[-1]['ha_streak'] < 0:
    for i in range(len(heiken_ashi) - 1, -1, -1):
        if heiken_ashi.iloc[i]['ha_streak'] > 0:
            print(f"First positive ha_streak from end found at index {i}")
            print(heiken_ashi.iloc[i])
            break


In [166]:
heiken_ashi.columns



Index(['time', 'ha_open', 'ha_high', 'ha_low', 'ha_close', 'ha_green',
       'ha_streak', 'ha_open_at_extreme'],
      dtype='object')

In [167]:
import numpy as np
import pandas as pd


def get_previous_swing(df: pd.DataFrame) -> dict | None:
    if df.empty:
        return None

    # ------------------------------------------------------------------
    # 1. Determine current trend (sign of the most-recent ha_streak)
    # ------------------------------------------------------------------
    current_streak = df.iloc[-1]['ha_streak']
    current_sign = np.sign(current_streak)
    print(f"Current ha_streak: {current_streak} (sign: {current_sign})")

    looking_for_positive = current_sign < 0  # we are in a down-trend → need last positive streak segment
    opposite_mask_val = 1 if looking_for_positive else -1
    print(f"Looking for {'positive' if looking_for_positive else 'negative'} streak segment...")

    first_opposite_idx = None
    for i in range(len(df) - 2, -1, -1):  # start from the bar *before* the last one
        sign_i = np.sign(df.iloc[i]['ha_streak'])
        if sign_i != current_sign:
            first_opposite_idx = i
            break
        pass

    print(f"First opposite streak index: {first_opposite_idx}", df.loc[first_opposite_idx])
    # ------------------------------------------------------------------
    # 2. Walk backward, collecting contiguous bars of the opposite sign
    # ------------------------------------------------------------------
    opposite_segment_idx = []

    for i in range(first_opposite_idx, -1, -1):  # start from the bar *before* the last one
        sign_i = np.sign(df.iloc[i]['ha_streak'])
        print("sign_i:", sign_i)
        if sign_i == opposite_mask_val:
            opposite_segment_idx.append(i)
        else:
            # we reached a bar that is *not* part of the opposite segment
            break

    print(f"Opposite segment indices: {opposite_segment_idx}")
    if not opposite_segment_idx:
        # No opposite-trend bars were found
        return None

    # ------------------------------------------------------------------
    # 3. Within that segment, find the extreme (min low / max high)
    # ------------------------------------------------------------------
    segment = df.loc[opposite_segment_idx]

    if looking_for_positive:  # current trend is down → swing high
        extreme_row = segment['ha_high'].idxmax()
        return {
            'type': 'swing_high',
            'index': int(extreme_row),
            'time': df.loc[extreme_row, 'time'],
            'price': float(df.loc[extreme_row, 'ha_high'])
        }
    else:  # current trend is up → swing low
        extreme_row = segment['ha_low'].idxmin()
        return {
            'type': 'swing_low',
            'index': int(extreme_row),
            'time': df.loc[extreme_row, 'time'],
            'price': float(df.loc[extreme_row, 'ha_low'])
        }


#--------------------------------------------------------------------------
# Example usage
#--------------------------------------------------------------------------
swing_info = get_previous_swing(heiken_ashi)
if swing_info is None:
    print("No opposite-trend segment found.")
else:
    print(f"Previous {swing_info['type']} at index {swing_info['index']} "
          f"({swing_info['time']}): {swing_info['price']}")

Current ha_streak: 1 (sign: 1)
Looking for negative streak segment...
First opposite streak index: 81 time                  2025-05-06 21:00:00+00:00
ha_open                              143.715778
ha_high                                 144.002
ha_low                                  142.423
ha_close                              143.18875
ha_green                                      0
ha_streak                                    -2
ha_open_at_extreme                            0
Name: 81, dtype: object
sign_i: -1
sign_i: -1
sign_i: 1
Opposite segment indices: [81, 80]
Previous swing_low at index 80 (2025-05-05 21:00:00+00:00): 142.354
